## 이 코드는 Andrej Karpathy님의 코드임을 밝힙니다.
출처: https://www.youtube.com/watch?v=kCc8FmEb1nY

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 32

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
with open('input.txt','r', encoding = 'utf-8') as f:
  text = f.read()

In [ ]:
print("Length of dataset in characters: ", len(text))

Length of dataset in characters:  1115394


In [ ]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l]) # 새로운 방식의 함수 정의법, lambda l 에서 l이 함수 입력값으로 받아짐.

print(encode("hii there"))
print(decode(encode("hii there")))


[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [ ]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [ ]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

여기서 training을 할 때, 모든 데이터를 한번에 넣어서 training을 하진 않는다. (계산 비용이 매우 높다.)
대신 랜덤하게 chunk를 뽑아내서 training에 각각 사용한다. (https://youtu.be/kCc8FmEb1nY?t=894)
여기서는 block_size라는 이름으로 maximum_length를 지정한다.

In [ ]:
# 길이 8의 훈련데이터를 설정하고 샘플링을 할 때는 길이 9를 샘플링 한다. 이는 autoregressive에서 흔하게 사용하는 teacher forcing 방식을 사용하기 위해 하는 것이다.
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
# 이제 입력값과 label값을 정의해보자
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


여기서, Transformer가 inference 하는 방식은 다음과 같다.
- 일단 block size 크기 전까지 계속 예측할 때는 이전의 예측 token들을 기반으로 다음 것을 예측한다.
- 그러나 block size만큼 예측을 하면, 그 후에는 truncate(이전 것을 잘라냄) 하면서 길이 8을 유지하며, 다음 토큰을 예측하는 식으로 예측이 진행된다.

여태까지는 single data에 대해서 데이터를 만든 것이였다.
이번에는 배치를 고려하며 다시 코드를 작성해보자.

In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size, )) # 마지막 8개(block_size=8)는 그 다음 것을 예측할 때 사용하지 못한다.

  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y


xb, yb = get_batch('train')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when input is {context.tolist()} the target: {target}")

when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53, 56, 1, 58, 46] the target: 39
when input is [44, 53, 56, 1, 58, 46, 39] the target: 58
when input is [44, 53, 56, 1, 58, 46, 39, 58] the target: 1
when input is [52] the target: 58
when input is [52, 58] the target: 1
when input is [52, 58, 1] the target: 58
when input is [52, 58, 1, 58] the target: 46
when input is [52, 58, 1, 58, 46] the target: 39
when input is [52, 58, 1, 58, 46, 39] the t

In [ ]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


## ※ 1. BigramLanguageModel 기반 문자열 생성

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337) # pytorch의 random seed 고정. 재구현성을 위해 설정

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.lm_head = nn.Linear(n_embed, vocab_size)


  def forward(self, idx, targets = None):
    # 여기서 idx와 targets의 shape는 (Batch size, Time(문장 하나의 단어 개수) ) 이다.
    tok_emb = self.token_embedding_table(idx) # 이것의 반환 결과 shape는 (Batch, Time, Channel(단어,토큰 하나의 임베딩 결과 벡터) ) 이다.
    # loss2 = F.cross_entropy(logits, targets) # 이 코드는 작동 안한다. 왜냐하면 F.cross entropy는 input을 (B,C,T)로 받기 때문이다. (https://youtu.be/kCc8FmEb1nY?t=1597)
    logits = self.lm_head(tok_emb)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C) # make sense한 shape임.
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(idx) # idx의 shape는 (B,T)이다.  여기서 self()는 forward를 부르는 함수라 한다. https://youtu.be/kCc8FmEb1nY?t=1886

      logits = logits[:, -1, :] # 예측한 것의 마지막 토큰(즉, 다음 예측, autoregressive model로 치면 바로 다음 예측한 토큰)에 대한 확률값만 가져온다.

      probs = F.softmax(logits, dim=-1) # softmax적용

      idx_next = torch.multinomial(probs, num_samples=1) # softmax로 나온 다음 토큰에 대한 확률을 기반으로 다음 토크을 뽑는다.

      idx = torch.cat((idx, idx_next), dim=1) # 기존 idx를 새로 뽑은 토큰과 concat해서 shape가 (B,T+1)인 idx를 새로 만든다.

    return idx # 매 스텝마다 sampling해서 token들이 concat된 최종 예측 sequence




m = BigramLanguageModel(vocab_size)
logits, loss  = m(xb, yb)
print(logits.shape, loss)
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65]) tensor(4.4922, grad_fn=<NllLossBackward0>)

lN!BJ'kysLCMFJPKOL?DP-QWwrEoL?jLDJQOL.f'RIHD'Hdhs Yv,wxatnscMZwtEOS'palkq3ssZeAvzF-QT;eMk;x.gQSFCLgx


현재는 bigram으로 앞의 하나 단어를 보고 다음 토큰을 유추하는 방식이다.

bigram model을 학습시켜보자.
(물론, 셰익스피어를 잘 학습하진 않았을 것이다. 하지만 어떠한 일정 패턴은 보일 것이다.)

In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(10000):

  xb, yb = get_batch("train")

  logits, loss = m(xb, yb)

  optimizer.zero_grad(set_to_none=True)
  # set_to_none 관련해서는 https://velog.io/@kjb0531/zerograd%EC%9D%98-%EC%9D%B4%ED%95%B4 참고

  loss.backward()

  optimizer.step()

  print(loss.item())

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
2.5015437602996826
2.4089622497558594
2.5160410404205322
2.4741573333740234
2.4618895053863525
2.422511339187622
2.4093828201293945
2.548694133758545
2.4375903606414795
2.4635019302368164
2.362743616104126
2.4755489826202393
2.357532024383545
2.376260280609131
2.444220542907715
2.4985249042510986
2.4815733432769775
2.5948750972747803
2.4701743125915527
2.4592843055725098
2.438476324081421
2.4719128608703613
2.575519561767578
2.4527246952056885
2.482231855392456
2.4283199310302734
2.522643804550171
2.531187057495117
2.4472765922546387
2.44993257522583
2.439965009689331
2.4108545780181885
2.4720044136047363
2.4077725410461426
2.530170440673828
2.5499625205993652
2.4030582904815674
2.3826732635498047
2.4314067363739014
2.5083019733428955
2.5102102756500244
2.5320401191711426
2.453016996383667
2.480130672454834
2.488076686859131
2.369173049926758
2.5427792072296143
2.441377639770508
2.5128674507141113
2.4079766273498535
2.57381272315979
2.463320255279541

In [ ]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=400)[0].tolist())) ###########


No t:
in ticothingh, much s ESThondiryrs y ve:

The thafenda merar
WAng, aothace Pay, ty---
Tu whea Why he Gome:
HIUSir.
SONI INGLor tco anoris thates ulifive hes thes, thacothanent! t male I w at pllar owifer.
F anthif.
BRII as s horugyouc-rar g wa
CagQUCEXle' ure ithe be bone chousse thely w, we tais RDWh e so
CARI SIRKINate: odelthr en tsthoond.
PUESe tamas he toly myo pe kereyenoidoo, jurderit


## ※ 2. Self-attention을 보기 전에 preview를 보자

여기 self attention에서 중요한 것은 t 스텝에서 그 이후 time step의 정보는 참고하면 안된다는 것이다. t 스텝의 토큰 예측에는 1,2,3,4, ... , t-1 스텝의 토큰만을 참고하도록 해야한다.

먼저 self attention의 preview로 이전 토큰들의 정보를 평균을 취하는 식으로 가져오는 것을 예제로써 해볼 것이다.
- 물론 평균을 취하면 정보손실이 심하여 실제로는 이렇게 하지 않고, 예시정도로 보면 된다.

In [ ]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
xbow = torch.zeros( (B,T,C))
for b in range(B):
  for t in range(T):
    xprev = x[b, :t+1]
    xbow[b, t] = torch.mean(xprev, 0)

여기서 box는 bag of word의 약자로, 단어의 순서를 고려하지 않고 단어의 출현 빈도만을 고려하여 텍스트 데이터를 표현하는 방법을 의미한다.
- bag of word 설명 출처: https://wikidocs.net/22650

In [ ]:
x[0] # 첫번째 배치에서 각 토큰마다의 channel(임베딩) 값

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [ ]:
xbow[0] # 첫번째 배치에 대한 각 time step마다의 average 값

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

**이번에는 위의 loop 두번 돌리는 것보다 더 효율적으로 만들어보자.**

먼저, lower triangular matrix를 만들어보자. 이는 pytorch에서 tril이라는 함수로 구현가능하다.



In [ ]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

이제 위의 이전 토큰만 고려한 평균 계산을 matrix 기반의 연산으로 수행해보자.
- a: 포지션 정보 (이전 토큰만 참고하도록 하기위한 lower triangular matrix)
- b: 토큰 정보
- c: bow 결과

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
b = torch.randint(0,10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('b=')
print(b)
print('c=')
print(c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


자, 일단 이전 토큰들의 평균은 아니지만, 이전 토큰만 고려해서 summation을 했다는 것을 통해 위의 for loop를 통한 이전 토큰만을 고려하는 메커니즘과 비슷하다는 것은 확인할 수 있다.
이제 summation이 아니라 평균을 계산하기 위해 코드를 수정해보자.

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a/torch.sum(a, dim=1, keepdim=True) # 평균 고려를 위해 추가된 코드,  keepdim 인자는 input과 동일한 size의 shape를 유지하도록 도와준다.
b = torch.randint(0,10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('b=')
print(b)
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


이제 동일한 결과를 내지만, 더 계산 효율적인 matrix연산을 통해서 xbow를 다시 계산해보자.

In [ ]:
# xbow ver 2: use tril function
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T,T) x (B,T,C)인데 여기서 B부분이 wei의 앞에 추가되서 (B,T,T) x (B, T, C) 형태의 multiplication이 된다. 이 곱의 결과는 (B,T,C) 가 된다.
print(torch.allclose(xbow2 ,xbow)) # torch.allclose 는 두 텐서가 동일한지 검사하는 함수.
print(xbow[0], xbow2[0]) # 이 둘의 결과가 일치함을 확인할 수 있다.


True
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]]) tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


- torch.allclose는 두 텐서가 동일한지 비교하는 데 사용할 수 있는 PyTorch 유틸리티 함수입니다. (https://runebook.dev/ko/docs/pytorch/generated/torch.allclose)

마지막은 mask_fill 기반의 연산이다.
- ※ 이 연산은 transformer 구현에서도 쓰이는 기법으로 알아두면 좋다.

In [ ]:
# xbow ver 3: use Softmax
tril = torch.tril(torch.ones(T,T))
print("tril: ", tril)
wei = torch.zeros((T,T))
print("wei, zeros: ", wei)
wei = wei.masked_fill(tril == 0, float('-Inf'))
print("wei, masked_fill: ", wei)
wei = F.softmax(wei, dim=-1)
print("wei, masked_fill softmax: ", wei)

xbow3 = wei @ x
torch.allclose(xbow, xbow3)


tril:  tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
wei, zeros:  tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
wei, masked_fill:  tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0.

True

version 3가 version 2보다 번거로워 보이는데, 왜 이 방법을 사용할까?
일단, 직관적으로 보면
1. 처음 wei, 즉 모든 값이 0인 wei는 서로간의 intercation strength로써 생각할 수 있는데, 처음에는 이전 token에 대해서 intercation은 0이라고 볼 수 있다.
2. wei.masked_fill(tril == 0, float('-Inf'))는 과거의 토큰들과는 communicate 할수 없도록 설정한 것이다. (https://youtu.be/kCc8FmEb1nY?t=3442)


이제 위에서 구현했던 BigramLanguageModel에 Self attention을 도입하여 Head라는 클래스와 함께 다시 구현해보자.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F


batch_size = 32
block_size = 8
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 32

torch.manual_seed(1337) # pytorch의 random seed 고정. 재구현성을 위해 설정

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed) # 이것 자체가 바로 logits가 안되고 이는 token embedding이 된다.   shape는 (B, T, C)
    self.position_embedding_table = nn.Embedding(block_size, n_embed) # 이는 position 값을 임베딩 하는 것인데, 여기서 중요한건 position의 차원 값이 token embedding 의 shape 크기와 같아야한다는 것이다.  shape는 (T, C) -> 이는 toekn embedding과 덧셈 연산을 할 때, (B, T, C) broadcast가 된다.
    self.sa_head = Head(n_embed)
    self.lm_head = nn.Linear(n_embed, vocab_size) # 이를 통해서 logits가 계산될 것이다.



  def forward(self, idx, targets = None):
    B,T = idx.shape

    # 여기서 idx와 targets의 shape는 (Batch size, Time(문장 하나의 단어 개수) ) 이다.
    tok_emb = self.token_embedding_table(idx) # 이것의 반환 결과 shape는 (Batch, Time, Channel(단어,토큰 하나의 임베딩 결과 벡터) ) 이다.
    pos_emb = self.position_embedding_table(torch.arrange(T, device = device)) # 0부터 T-1 값에 대해서 (입력 sequnece) 이 위치(Time step)값들에 대한 position embedding값을 반환한다.
    x = tok_emb + pos_emb # 이것이 최종 입력값이 된다. token에 position을 고려한 input
    x = self.sa_head(x)

    # toekn + pos_emb를 했을 떄, 아직까진 useful하지 않은데, 왜냐하면 bigram 모델이기 때문이다.

    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C) # make sense한 shape
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):

      idx_cond = idx[:, -block_size:] # 마지막 block size 토큰의 idx를 crop (https://youtu.be/kCc8FmEb1nY?t=4851)

      logits, loss = self(idx_cond) # idx의 shape는 (B,T)이다.

      logits = logits[:, -1, :]

      probs = F.softmax(logits, dim=-1)

      idx_next = torch.multinomial(probs, num_samples=1) # softmax로 나온 다음 토큰에 대한 확률을 기반으로 다음 토크을 뽑는다.

      idx = torch.cat((idx, idx_next), dim=1) # 기존 idx를 새로 뽑은 토큰과 concat해서 shape가 (B,T+1)인 idx를 새로 만든다.

    return idx # 매 스텝마다 sampling해서 token들이 concat된 최종 예측 sequence




m = BigramLanguageModel(vocab_size)
logits, loss  = m(xb, yb)
print(logits.shape, loss)
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

## ※ 3. 이제부터 self attention을 보자.
https://youtu.be/kCc8FmEb1nY?t=3718

- self attention에서 self의 의미는 key와 query, value가 모두 같은 source로부터 나왔다는 것을 의미한다. (여기서는 x라는 동일한 source에서 나온다.)

먼저, single head인 작은 self attention을 구현해보자.

In [ ]:
# version 4: self attention
# 1. 여기서 달라진 점:
# 이전까지는 그저 이전 토큰들의 평균을 취하였다. 여기서는 attention을 적용해서 본다.
# 지금은 data dependent하게 보려고 한다. 예: 현재 토큰에 모음이 있다는 것은 그 앞에 토큰을 집중할 때 자음을 더 집중해서 봤다는 의미로 볼 수 있다.

# 먼저 query와 key에 대해서 구현할 것이다.
# query: 우리가 집중해서 보려고 하는 것
# key: 각 토큰이 가지고 있는 content

# 2. 이들을 dot product를 할 것이다. 만약 값이 높게 나온다면 해당 token의 query는 해당 토큰의 key에 집중해서 다음 것을 예측한다고 볼 수 있다.

torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias = False)
query = nn.Linear(C, head_size, bias = False)

k = key(x)   # shape: (B, T, 16)
q = query(x) # shape: (B, T, 16)
# 3. 여기까지, 현재 모든 batch의 각각 token에 대해서 key와 query를 생성했다. 아직 token간의 interaction (self attention)은 계산되지 않았다.

wei = q @ k.transpose(-2, -1) * C**-0.5 # 4. k.transpose: 그냥 .T를 하면 batch dimension고려가 힘들다. 저렇게 지정하면 (B, T, 16) @ (B, 16, T) 형식의 곱이 가능하다.   마지막의 C**-0.5는 head_size의 square root를 곱해서 scaled attention을 만드는 것이다.
# wei shape: (B, T, T)

# 5. 이제 value라는 것을 만들자. 기존에는 wei @ x로 토큰 자체를 attention에 곱하는 형태였다.

value = nn.Linear(C, head_size, bias = False)



tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-Inf')) # 만약 encoder block이였다면 모든 토큰들이 서로 communicate를 해야되서 이 코드가 없었을 것이다. 하지만 decoder는 autoregressive하게 작동해야해서 이 부분을 넣었다.
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v # 이 v가 aggrigate하는 것이다.

out.shape

torch.Size([4, 8, 16])

이제 data dependent하게 출력한 이전 토큰들을 고려하는 aggregation weight를 보자.

In [ ]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4264, 0.5736, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3151, 0.3022, 0.3827, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3007, 0.2272, 0.2467, 0.2253, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1635, 0.2048, 0.1776, 0.1616, 0.2926, 0.0000, 0.0000, 0.0000],
         [0.1403, 0.2272, 0.1454, 0.1244, 0.2678, 0.0949, 0.0000, 0.0000],
         [0.1554, 0.1815, 0.1224, 0.1213, 0.1428, 0.1603, 0.1164, 0.0000],
         [0.0952, 0.1217, 0.1130, 0.1453, 0.1137, 0.1180, 0.1467, 0.1464]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4300, 0.5700, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3379, 0.2559, 0.4061, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2867, 0.2188, 0.2786, 0.2159, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1297, 0.1813, 0.1683, 0.2990, 0.2217, 0.0000, 0.0000, 0.0000],
         [0.1584, 0.167

이전처럼 평균을 취한 것과 다르게, 모든 배치사이에 weight가 같지 않다.
- x에서의 각각의 batch에 대한 값들이 다르기 때문


scaled atttention을 보자.

In [ ]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1)

In [ ]:
k.var(), q.var(), wei.var()

(tensor(0.9487), tensor(1.0449), tensor(14.3682))

variance가 크게 나왔다.

이번엔 head의 square root를 곱해보자.

In [ ]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [ ]:
k.var(), q.var(), wei.var()

(tensor(0.9416), tensor(1.0104), tensor(1.0879))

variance가 매우 줄었다.

attention의 scaling이 중요한 이유는, 우리가 attention의 마지막에 softmax를 적용하기 때문이다. variance가 크다는 말은 벡터 안의 값들 차이가 매우 크다는 의미이다. 너무 negative로 쳐져있거나 postive값이 큰 원소값 하나가 있을것인데, 그렇게 하면 softmax후에 one hot vector 형태의 결과가 나올 것이다.

https://youtu.be/kCc8FmEb1nY?t=4778 파이토치에서 buffer의 의미

Layer norm은 batch norm과 비슷하다.
차이점: batch norm은 배치단위로, dim = 0 에 대해서 normalize하면, Layer norm은 feature 단위로 dim=1에 대해서 normalize 한다.
- https://youtu.be/kCc8FmEb1nY?t=5594
- Batch size에 대해 robust 하다.

## 4. self attention을 고려하여, Head 클래스를 추가한 새로운 버전의 Model을 다시 보자.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F


batch_size = 32
block_size = 8
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 32


import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337) # pytorch의 random seed 고정. 재구현성을 위해 설정





# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


# estimate_loss는 깊게 안보고 가져온 코드이다.
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out



class Head(nn.Module):  # 4. 에서 추가된 부분
  ### 이 클래스는 하나의 셀프 어텐션 head 단위이다.
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embed, head_size, bias = False)
    self.query = nn.Linear(n_embed, head_size, bias = False)
    self.value = nn.Linear(n_embed, head_size, bias = False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    # register buffer는 gradients를 요구하지 않아서 parameters로써 저장되지 않고 state_dict로 저장된다. 이는 batch norm 레이어의 mean이나 std의 tracking에 용이하다 한다.
    # 파이토치에서 state dict는 각 parameter tensor의 레이어에 매핑하는 dictionary object이다.
    # Buffer는 Module에 저장해 놓고 사용하는 Tensor의 일종으로 학습을 통해 계산되지 않는 Tensor입니다. (https://teamdable.github.io/techblog/PyTorch-Module)



  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x)
    q = self.key(x)

    wei = q @ k.transpose(-2,-1) * C**-0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-Inf'))
    wei = F.softmax(wei, dim = -1)

    v = self.value(x)

    out = wei @ v # wei는 하나의 row가 하나의 query에 대한 각각의 key에 대한 attention,   v는 하나의 row가 하나의 단어에 대한 value 변환 값. 이 연산을 하면 각 row에 각 attention이 곱해져서 row에 대해서 더해지는 것
    return out





class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    self.sa_head = Head(n_embed)
    self.lm_head = nn.Linear(n_embed, vocab_size)



  def forward(self, idx, targets = None):
    B,T = idx.shape

    # 여기서 idx와 targets의 shape는 (Batch size, Time(문장 하나의 단어 개수) ) 이다.
    tok_emb = self.token_embedding_table(idx) # 이것의 반환 결과 shape는 (Batch, Time, Channel(단어,토큰 하나의 임베딩 결과 벡터) ) 이다.
    pos_emb = self.position_embedding_table(torch.arange(T, device = device)) # 0부터 T-1 값에 대해서 (입력 sequnece) 이 위치(Time step)값들에 대한 position embedding값을 반환한다.
    x = tok_emb + pos_emb # 이것이 최종 입력값이 된다. token에 position을 고려한 input
    x = self.sa_head(x)


    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):

      idx_cond = idx[:, -block_size:]

      logits, loss = self(idx_cond) # idx의 shape는 (B,T)이다.  여기서 self()는 forward를 부르는 함수라 한다. https://youtu.be/kCc8FmEb1nY?t=1886

      logits = logits[:, -1, :] # 예측한 것의 마지막 토큰(즉, 다음 예측, autoregressive model로 치면 바로 다음 예측한 토큰)에 대한 확률값만 가져온다.

      probs = F.softmax(logits, dim=-1) # softmax적용

      idx_next = torch.multinomial(probs, num_samples=1) # softmax로 나온 다음 토큰에 대한 확률을 기반으로 다음 토크을 뽑는다.

      idx = torch.cat((idx, idx_next), dim=1) # 기존 idx를 새로 뽑은 토큰과 concat해서 shape가 (B,T+1)인 idx를 새로 만든다.

    return idx # 매 스텝마다 sampling해서 token들이 concat된 최종 예측 sequence



model = BigramLanguageModel(vocab_size)
m = model.to(device)
logits, loss  = m(xb, yb)
print(logits.shape, loss)
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([256, 65]) tensor(4.2735, grad_fn=<NllLossBackward0>)

uoabMfqMY.ukrspu,Xvy!CJFfmMSgS.JCnOEmygGA&-SPlLyuGYfGXMaInv'Z,wX$,OMdsR!vHQImqSSPe;N
'?.z;?UAfCAufX:


In [ ]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)
optimizer = torch.optim.AdamW(m.parameters(), learning_rate)

In [ ]:
batch_size = 32
for steps in range(max_iters):

  if steps % eval_interval == 0 or steps == max_iters - 1:
      losses = estimate_loss()
      print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batch("train")

  logits, loss = m(xb, yb)

  optimizer.zero_grad(set_to_none=True)
  # set_to_none 관련해서는 https://velog.io/@kjb0531/zerograd%EC%9D%98-%EC%9D%B4%ED%95%B4 참고

  loss.backward()

  optimizer.step()

step 0: train loss 4.2507, val loss 4.2519
step 500: train loss 2.6251, val loss 2.6253
step 1000: train loss 2.5230, val loss 2.5281
step 1500: train loss 2.4835, val loss 2.4622
step 2000: train loss 2.4540, val loss 2.4539
step 2500: train loss 2.4422, val loss 2.4394
step 3000: train loss 2.4219, val loss 2.4474
step 3500: train loss 2.4167, val loss 2.4377
step 4000: train loss 2.4135, val loss 2.4303
step 4500: train loss 2.3945, val loss 2.4267
step 4999: train loss 2.4155, val loss 2.4202


In [ ]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=400)[0].tolist()))


Af blost onetecal toumur stheme rderxlesh sble:--
ABe cthal sk molitha ve qutnoppland Pso Maindo of scepary he hath merer corginishueak.
Whierd stiowome alllet,
L;
ble gs t y teil monend dfild; bandil avendinese deane
Edild, ssat ndervey hilleme ge Yond thus.

Ye heneres at,
S:
My, so imough thor ss yocut te?

LIRMORDUS:
PI chame Rerenetrgo yet;
My byok oro aut hye tthouther sstom, I cker, halour 


## 5. 이제 Head를 여러 개를 추가하는 Multi head attention을 구현해서 다시 모델을 만들어보자.

이는 그저 attention head를 parallel하게 여러 개를 추가한 것이다.
Multi head를 추가했을 때의 이점은 다양한 관점에서의 attention 해석을 할 수있게 될 것이다.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F


batch_size = 32
block_size = 8
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 32


import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337) # pytorch의 random seed 고정. 재구현성을 위해 설정





# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class MultiHeadAttention(nn.Module): # 5. 에서 추가된 부분
  def __init__(self, num_heads, head_size): # head_size는 각각 head에서의 size를 의미하는 것이다. 맨 처음에 총 크기를 의미하는게 아니라 총 크기에서 head 개수만큼으로 나눠준 것을 의미
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) # nn.Modulelist는 Python의 list 타입 데이터 안에 원소 형태로 저장된 각각의 layer를 저장하는 것을 의미한다.
    # nn.ModuleList는 nn.Sequential처럼 안에 데이터 넣는다고 바로 forwrad가 작동해서 순차적으로 하는 것이 아니다. 이것은 modulelist안에 있는 각각의 layer를 불러와서 안에 데이터를 넣는식으로 진행해야 한다.
    # Multi head attention은 병렬적으로 계산해야 되는거다. Head 1과 Head 2는 서로 간섭하는 연산을 안한다.
    # 따라서, 아래 forward처럼 각각의 head마다 입력인 x를 넣어서 각각의 head 결과들을 concat 하는 것이다.

  def forward(self, x):
    return torch.cat([h(x) for h in self.heads], dim=-1) # dim = -1은 channel에 대해서 concat을 하겠다는 의미


class Head(nn.Module):
  ### 이 클래스는 하나의 셀프 어텐션 head 단위이다.
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embed, head_size, bias = False)
    self.query = nn.Linear(n_embed, head_size, bias = False)
    self.value = nn.Linear(n_embed, head_size, bias = False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))




  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x)
    q = self.key(x)

    wei = q @ k.transpose(-2,-1) * C**-0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-Inf'))
    wei = F.softmax(wei, dim = -1)

    v = self.value(x)

    out = wei @ v
    return out





class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    self.sa_head = MultiHeadAttention(4, n_embed//4)
    self.lm_head = nn.Linear(n_embed, vocab_size)



  def forward(self, idx, targets = None):
    B,T = idx.shape

    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device = device))
    x = tok_emb + pos_emb t
    x = self.sa_head(x)



    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):

      idx_cond = idx[:, -block_size:]



      logits, loss = self(idx_cond)

      logits = logits[:, -1, :]

      probs = F.softmax(logits, dim=-1)

      idx_next = torch.multinomial(probs, num_samples=1)

      idx = torch.cat((idx, idx_next), dim=1)

    return idx




model = BigramLanguageModel(vocab_size)
m = model.to(device)
optimizer = torch.optim.AdamW(m.parameters(), learning_rate)

batch_size = 32
for steps in range(max_iters):

  if steps % eval_interval == 0 or steps == max_iters - 1:
      losses = estimate_loss()
      print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batch("train")

  logits, loss = m(xb, yb)

  optimizer.zero_grad(set_to_none=True)

  loss.backward()

  optimizer.step()


print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=2000)[0].tolist()))

step 0: train loss 4.2324, val loss 4.2307
step 500: train loss 2.6198, val loss 2.6378
step 1000: train loss 2.4783, val loss 2.4858
step 1500: train loss 2.4226, val loss 2.4245
step 2000: train loss 2.3780, val loss 2.3861
step 2500: train loss 2.3597, val loss 2.3673
step 3000: train loss 2.3354, val loss 2.3498
step 3500: train loss 2.3119, val loss 2.3289
step 4000: train loss 2.3097, val loss 2.2979
step 4500: train loss 2.3003, val loss 2.3063
step 4999: train loss 2.2951, val loss 2.3055

Shiser:
We mallokier sorestrarl thorn hit tavoefars'll lost ins, ingd riee thedroee o, lol be hom da


4번의 loss 값인 2.4202에 비해서, multi head attention을 추가하니 2.3 정도로 줄었다.

In [ ]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=2000)[0].tolist()))


CHENE:
Aou tle.

Yan searly dome thall;
On;
Srienanged dollf!
Anut I a sowol: wountret bllanthre hand four hely tho, you,
Ahald tho sof quneeay
Serve quwee: he grue hond:

Tharesed ume doud thod Is
BCUCKARe ghour by beate:
Gh; hirts,
If, thee---
HOP-loth Rallyoo,
Is ther diss ipne; sho nouprepear dullesn.

Gee se?
Nw;
But bar:
Waupt thigeretece, do;
And hald, oungreel oildspt nre lorshndace,
Gugec-:ater hadbarensit by thad lave goio pame mil?
Whensee he pos I wary.

Clit weea-sto bem'ss ait you to as thay thise my that onometteeed? ou iwiro cend out, hithinst dous be me; and
A'lscr'd fre;
Whoulsech lace, youdn,
Whee the mar lock at, then, hit;
Why?

ARA! le inly,
I'rier thelaver owee too bedotlet gouv
Mand fts alve lost yeresto me ithep iciecameak ing as forst he meet! owne? wit.

TRotegine for, sall woene erut aster, shatard, to, Gol, the my;
Macy, fring whim fby Hete dlicpeeadicon ant ft to bat aer alene?


Tarm: tis art hy Lornd Colrouce notle lonocar't tocus
parceang dis,
B:
If me

## 6. Position-wise Feed forward neural network를 추가한다.

우리가 multi head self attention을 하고 나서, logits(토큰 에측)을 계산하는 것은 너무 빠르다. 각 어텐션 head에서 찾은 정보를 좀 더 가공하는 계산도 필요하다.
추가적으로 non linearity도 추가할 수 있다.
- ChatGPT에서 받은 답변


그게 FFNN의 역할이다.

FFNN은 토큰 단위로 넣어서 이루어진다. 즉, 하나의 문장에서 단어 하나하나가 FFNN에 넣어서 계산된다는 의미이다.(https://youtu.be/kCc8FmEb1nY?t=5162)

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F


batch_size = 32
block_size = 8
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 32


import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337) # pytorch의 random seed 고정. 재구현성을 위해 설정





# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y



@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

  def forward(self, x):
    return torch.cat([h(x) for h in self.heads], dim=-1)


class Head(nn.Module):

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embed, head_size, bias = False)
    self.query = nn.Linear(n_embed, head_size, bias = False)
    self.value = nn.Linear(n_embed, head_size, bias = False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))




  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x)
    q = self.key(x)

    wei = q @ k.transpose(-2,-1) * C**-0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-Inf'))
    wei = F.softmax(wei, dim = -1)

    v = self.value(x)

    out = wei @ v
    return out




class FeedForward(nn.Module): # 6. 에서 추가된 부분
  def __init__(self, n_embed):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embed, n_embed),
        nn.ReLU()
    )

  def forward(self, x):
    return self.net(x)





class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed) #
    self.position_embedding_table = nn.Embedding(block_size, n_embed) #
    self.sa_head = MultiHeadAttention(4, n_embed//4)
    self.ffwd = FeedForward(n_embed)
    self.lm_head = nn.Linear(n_embed, vocab_size)



  def forward(self, idx, targets = None):
    B,T = idx.shape

    tok_emb = self.token_embedding_table(idx) #
    pos_emb = self.position_embedding_table(torch.arange(T, device = device)) #
    x = tok_emb + pos_emb #
    x = self.sa_head(x)
    x = self.ffwd(x)

    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C) # make sense한 shape임.
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):

      idx_cond = idx[:, -block_size:]

      logits, loss = self(idx_cond)

      logits = logits[:, -1, :]

      probs = F.softmax(logits, dim=-1) #

      idx_next = torch.multinomial(probs, num_samples=1) #

      idx = torch.cat((idx, idx_next), dim=1) #

    return idx




model = BigramLanguageModel(vocab_size)
m = model.to(device)
optimizer = torch.optim.AdamW(m.parameters(), learning_rate)

batch_size = 32
for steps in range(max_iters):

  if steps % eval_interval == 0 or steps == max_iters - 1:
      losses = estimate_loss()
      print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batch("train")

  logits, loss = m(xb, yb)

  optimizer.zero_grad(set_to_none=True)

  loss.backward()

  optimizer.step()


print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=2000)[0].tolist()))

step 0: train loss 4.1982, val loss 4.1985
step 500: train loss 2.5810, val loss 2.5915
step 1000: train loss 2.4613, val loss 2.4685
step 1500: train loss 2.4047, val loss 2.4045
step 2000: train loss 2.3512, val loss 2.3667
step 2500: train loss 2.3299, val loss 2.3504
step 3000: train loss 2.3239, val loss 2.3340
step 3500: train loss 2.2909, val loss 2.3129
step 4000: train loss 2.2908, val loss 2.2904
step 4500: train loss 2.2730, val loss 2.2854
step 4999: train loss 2.2628, val loss 2.2914

ARECARDHHA:
Yod
Thrier the aved of ingooid botlet gous
Mand fts all pa,
Dout would co thep sciecledakniceraing sor he meet! wome? wondeckitteg no fo, pot-lerrene eyom aster, shatard, to, bol, the my;
Macy, frand whir funt.

Th licpen dicong haugt to bat ait alend?


Tare: thirk to!

Lornd lolde sean tou lonocar!
Faccis: anceang dis,
Thanfemeed youh no co dereirn troug sst thee sle
Thoust.

Ad it my gell reaboping to owisie, falll.

Bus aund is Hortheor mas pes queave tuth nohe st: wit,
Rorner

5. 의 loss인 2.3 비해서 loss가 2.29까지 줄었다.
- 영상에서도 2.28에서 2.24로 줄어, 감소량이 그렇게 크진 않았다.

## 7. 여기서는 성능 개선이 아닌, Decoder block을 쌓을 때 더 편하게 쌓고자 여태까지의 연산을 하나의 class로 묶어서 하나의 transformer block으로 만들고자 한다.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F


batch_size = 32
block_size = 8
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 32


import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337) # pytorch의 random seed 고정. 재구현성을 위해 설정





# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

  def forward(self, x):
    return torch.cat([h(x) for h in self.heads], dim=-1)


class Head(nn.Module):

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embed, head_size, bias = False)
    self.query = nn.Linear(n_embed, head_size, bias = False)
    self.value = nn.Linear(n_embed, head_size, bias = False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))



  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x)
    q = self.key(x)

    wei = q @ k.transpose(-2,-1) * C**-0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-Inf'))
    wei = F.softmax(wei, dim = -1)

    v = self.value(x)

    out = wei @ v
    return out




class FeedForward(nn.Module)
  def __init__(self, n_embed):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embed, n_embed),
        nn.ReLU()
    )

  def forward(self, x):
    return self.net(x)


class Block(nn.Module): # 7. 에서 추가된 부분
  def __init__(self, n_embed, n_head):
    super().__init__()
    head_size = n_embed // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embed)

  def forward(self, x):
    x = self.sa(x)
    x = self.ffwd(x)
    return x



class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    self.blocks = nn.Sequential( # 7. 에서 추가된 부분
        Block(n_embed, n_head=4),
        Block(n_embed, n_head=4),
        Block(n_embed, n_head=4)
    )
    self.lm_head = nn.Linear(n_embed, vocab_size)



  def forward(self, idx, targets = None):
    B,T = idx.shape

    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device = device))
    x = tok_emb + pos_emb
    x = self.blocks(x) # 7. 에서 추가된 부분


    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):

      idx_cond = idx[:, -block_size:]

      logits, loss = self(idx_cond)

      logits = logits[:, -1, :]

      probs = F.softmax(logits, dim=-1)

      idx_next = torch.multinomial(probs, num_samples=1)

      idx = torch.cat((idx, idx_next), dim=1)

    return idx




model = BigramLanguageModel(vocab_size)
m = model.to(device)
optimizer = torch.optim.AdamW(m.parameters(), learning_rate)

batch_size = 32
for steps in range(max_iters):

  if steps % eval_interval == 0 or steps == max_iters - 1:
      losses = estimate_loss()
      print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batch("train")

  logits, loss = m(xb, yb)

  optimizer.zero_grad(set_to_none=True)

  loss.backward()

  optimizer.step()


print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=2000)[0].tolist()))

step 0: train loss 4.2116, val loss 4.2077
step 500: train loss 2.9879, val loss 2.9910
step 1000: train loss 2.6431, val loss 2.6352
step 1500: train loss 2.5289, val loss 2.5233
step 2000: train loss 2.4537, val loss 2.4840
step 2500: train loss 2.4292, val loss 2.4298
step 3000: train loss 2.3800, val loss 2.4108
step 3500: train loss 2.3687, val loss 2.3904
step 4000: train loss 2.3494, val loss 2.3482
step 4500: train loss 2.3178, val loss 2.3407
step 4999: train loss 2.3009, val loss 2.3288

In- myouand wile me fin the she!

 a nole urrordsoste hof greide of wis hacke sits at wat'
Wec!

'w':
Haire
Whuthlive sowe fator.

RCHUWEUCIOLLTS:
Een net hat Lfit nok art,
Hhasth beirint sas of come walle!
Bons wour
nesssis lis be you win say'ders to bilimensk avand chal shhat it willign lit atherit:
My
Sins mouse.

WIIOSISIOOORECEE:
Bince
Innd, perasy' cray cos this, the, me:
got come whiurt mee heall, thour unos dongerer; mod hid, groanf to hoe cone!':
Ye, sivermou, bon dit bringacecis the

## 8. Block을 추가함에 따른 깊어진 Neural Net을 더 잘 optimize하기 위해 skip connection 추가

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F


batch_size = 32
block_size = 8
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 32


import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)





# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):

    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y



@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out



class Head(nn.Module):

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embed, head_size, bias = False)
    self.query = nn.Linear(n_embed, head_size, bias = False)
    self.value = nn.Linear(n_embed, head_size, bias = False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))




  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x)
    q = self.key(x)

    wei = q @ k.transpose(-2,-1) * C**-0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-Inf'))
    wei = F.softmax(wei, dim = -1)

    v = self.value(x)

    out = wei @ v
    return out



class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    self.proj = nn.Linear(n_embed, n_embed) # 8. 에서 추가된 부분
    # https://youtu.be/kCc8FmEb1nY?t=5457

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.proj(out) # 8. 에서 추가된 부분
    return out



class FeedForward(nn.Module):
  def __init__(self, n_embed):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embed, 4*n_embed),
        nn.ReLU(),
        nn.Linear(4*n_embed, n_embed) # 8. 에서 추가된 부분
    )

  def forward(self, x):
    return self.net(x)


class Block(nn.Module):
  def __init__(self, n_embed, n_head):
    super().__init__()
    head_size = n_embed // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embed)

  def forward(self, x):
    x = x + self.sa(x)  # 8. 에서 추가된 부분
    x = x + self.ffwd(x)  # 8. 에서 추가된 부분
    return x



class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    self.blocks = nn.Sequential( # 7. 에서 추가된 부분
        Block(n_embed, n_head=4),
        Block(n_embed, n_head=4),
        Block(n_embed, n_head=4)
    )
    self.lm_head = nn.Linear(n_embed, vocab_size)



  def forward(self, idx, targets = None):
    B,T = idx.shape

    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device = device))
    x = tok_emb + pos_emb
    x = self.blocks(x)

    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):

      idx_cond = idx[:, -block_size:]

      logits, loss = self(idx_cond)

      logits = logits[:, -1, :]

      probs = F.softmax(logits, dim=-1)

      idx_next = torch.multinomial(probs, num_samples=1)

      idx = torch.cat((idx, idx_next), dim=1)

    return idx



model = BigramLanguageModel(vocab_size)
m = model.to(device)
optimizer = torch.optim.AdamW(m.parameters(), learning_rate)

print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')
batch_size = 32
for steps in range(max_iters):

  if steps % eval_interval == 0 or steps == max_iters - 1:
      losses = estimate_loss()
      print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batch("train")

  logits, loss = m(xb, yb)

  optimizer.zero_grad(set_to_none=True)

  loss.backward()

  optimizer.step()


print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=2000)[0].tolist()))

0.041921 M parameters
step 0: train loss 4.6347, val loss 4.6314
step 500: train loss 2.4105, val loss 2.4048
step 1000: train loss 2.3186, val loss 2.3166
step 1500: train loss 2.2389, val loss 2.2510
step 2000: train loss 2.2065, val loss 2.2372
step 2500: train loss 2.1707, val loss 2.2072
step 3000: train loss 2.1409, val loss 2.1956
step 3500: train loss 2.1274, val loss 2.1695
step 4000: train loss 2.0951, val loss 2.1599
step 4500: train loss 2.0676, val loss 2.1487
step 4999: train loss 2.0634, val loss 2.1314

Row dut
Me theer?

VISANDY:
No adl ye: by chom, I sall you ono the
Sho lord:
Hatishad, bowl for the that a magkn--dogry, bebed thouses of these wamis, I Hear dradond in eif heather of ally, facce,
To mockn youd your, muds be geapaucks ofe bock, I omme a gokild, same the mone.

IC be had wash engroy peatize duporen so; hay all, the by be and de pustes for light coldan'd grever
Sell to wey chound me or the all hum docincs and I a copar'ds: besees mordse have, craice: he at

6. 의 2.29에서  2.23까지 줄었다.

## 9. Block을 추가함에 따른 깊어진 Neural Net을 더 잘 optimize하기 위해 Layer Normalization 추가

https://youtu.be/kCc8FmEb1nY?t=5560

단, 여기서는 원본 Transformer 논문과는 다르게 Layer normalization의 위치가 skip connection과 같이 있는 것이 아닌, Multi head attention, FFNN에 들어가기 전으로 바꾼다.
- 원본 Transformer의 경우를 post norm, 여기서 구현하는 것을 pre norm이라 하는데, 요즘은 pre norm이 더 좋다 하여 이 방법을 많이 쓴다고 한다.

In [ ]:
# 직접 작성해본 코드는 아니고, 가져온 것인데 사용하진 않았다.
# nn.LayerNorm를 대신 사용

class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F


batch_size = 32
block_size = 8
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 32


import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)





# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):

    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embed, head_size, bias = False)
    self.query = nn.Linear(n_embed, head_size, bias = False)
    self.value = nn.Linear(n_embed, head_size, bias = False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))



  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x)
    q = self.key(x)

    wei = q @ k.transpose(-2,-1) * C**-0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-Inf'))
    wei = F.softmax(wei, dim = -1)

    v = self.value(x)

    out = wei @ v
    return out



class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    self.proj = nn.Linear(n_embed, n_embed)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.proj(out)
    return out



class FeedForward(nn.Module):
  def __init__(self, n_embed):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embed, 4*n_embed),
        nn.ReLU(),
        nn.Linear(4*n_embed, n_embed)
    )

  def forward(self, x):
    return self.net(x)


class Block(nn.Module):
  def __init__(self, n_embed, n_head):
    super().__init__()
    head_size = n_embed // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embed)
    self.ln1 = nn.LayerNorm(n_embed) # 9. 에서 추가된 부분
    self.ln2 = nn.LayerNorm(n_embed) # 9. 에서 추가된 부분
    # Layer norm은 batch가 아니라 하나의 sample의 feature의 개수에 대해서 normalization을 하기 때문에 feature의 개수인 n_embed를 입력으로 넣어준다.

  def forward(self, x):
    x = x + self.sa(self.ln1(x))  # 8. 9. 에서 추가된 부분
    x = x + self.ffwd(self.ln2(x))  # 8. 9. 에서 추가된 부분
    return x



class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    self.blocks = nn.Sequential(
        Block(n_embed, n_head=4),
        Block(n_embed, n_head=4),
        Block(n_embed, n_head=4),
        nn.LayerNorm(n_embed)
    )
    self.lm_head = nn.Linear(n_embed, vocab_size)



  def forward(self, idx, targets = None):
    B,T = idx.shape

    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device = device))
    x = tok_emb + pos_emb
    x = self.blocks(x)

    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):

      idx_cond = idx[:, -block_size:]

      logits, loss = self(idx_cond)

      logits = logits[:, -1, :]

      probs = F.softmax(logits, dim=-1)

      idx_next = torch.multinomial(probs, num_samples=1)

      idx = torch.cat((idx, idx_next), dim=1)

    return idx




model = BigramLanguageModel(vocab_size)
m = model.to(device)
optimizer = torch.optim.AdamW(m.parameters(), learning_rate)

print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')
batch_size = 32
for steps in range(max_iters):

  if steps % eval_interval == 0 or steps == max_iters - 1:
      losses = estimate_loss()
      print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batch("train")

  logits, loss = m(xb, yb)

  optimizer.zero_grad(set_to_none=True)

  loss.backward()

  optimizer.step()


print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=2000)[0].tolist()))

0.042369 M parameters
step 0: train loss 4.3112, val loss 4.3103
step 500: train loss 2.4357, val loss 2.4328
step 1000: train loss 2.3157, val loss 2.3159
step 1500: train loss 2.2235, val loss 2.2397
step 2000: train loss 2.1968, val loss 2.2259
step 2500: train loss 2.1424, val loss 2.1783
step 3000: train loss 2.1217, val loss 2.1692
step 3500: train loss 2.1012, val loss 2.1451
step 4000: train loss 2.0810, val loss 2.1387
step 4500: train loss 2.0562, val loss 2.1336
step 4999: train loss 2.0383, val loss 2.1033

Row dut
Mofth eread?
What pra't our that the to velacEth by to thy
Should she atce addon, last be hath light on this that with thour:
So sould a notid I Hattior, for in enfores leajok aloys fromffers mocke youd, would thou to eapauces ofe bock.
I ome
The to lougse the ploower so, be hath howse shald basign dumper,
But hay all, the by be andede a my sif.

JULABES:
She thule emutel,
To Vurdews fill bodde?

NORIV:
Thech bs anguter, oparyis:
To neags sold have, Or, bake maat

8.에서의 loss인 2.2379에 비해서 여기서는 loss가 2.1 까지 줄었다.

여기까지가 Transformer 원문의 Decoder 파트였다.

## 10. Dropout 추가 및 마지막 코드 정리

https://youtu.be/kCc8FmEb1nY?t=5888 dropout 언제 추가하는지.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F


batch_size = 32
block_size = 8
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 32
n_layers = 3  # 9. 에서 추가된 부분
dropout = 0.2 # 10. 에서 추가된 부분

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337) # pytorch의 random seed 고정. 재구현성을 위해 설정





# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embed, head_size, bias = False)
    self.query = nn.Linear(n_embed, head_size, bias = False)
    self.value = nn.Linear(n_embed, head_size, bias = False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))


  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x)
    q = self.key(x)

    wei = q @ k.transpose(-2,-1) * C**-0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-Inf'))
    wei = F.softmax(wei, dim = -1)

    v = self.value(x)

    out = wei @ v
    return out



class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    self.proj = nn.Linear(n_embed, n_embed)
    self.dropout = nn.Dropout(dropout) # 10. 에서 추가된 부분


  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out))
    return out



class FeedForward(nn.Module):
  def __init__(self, n_embed):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embed, 4*n_embed),
        nn.ReLU(),
        nn.Linear(4*n_embed, n_embed),
        nn.Dropout(dropout)
    )

  def forward(self, x):
    return self.net(x)


class Block(nn.Module):
  def __init__(self, n_embed, n_head):
    super().__init__()
    head_size = n_embed // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embed)
    self.ln1 = nn.LayerNorm(n_embed)
    self.ln2 = nn.LayerNorm(n_embed)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x



class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    self.blocks = nn.Sequential(
        *[Block(n_embed, n_head = n_head) for _ in range(n_layers)]

    )
    self.lm_head = nn.Linear(n_embed, vocab_size)


  def forward(self, idx, targets = None):
    B,T = idx.shape

    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device = device))
    x = tok_emb + pos_emb
    x = self.blocks(x)



    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):

      idx_cond = idx[:, -block_size:]



      logits, loss = self(idx_cond)

      logits = logits[:, -1, :]

      probs = F.softmax(logits, dim=-1)

      idx_next = torch.multinomial(probs, num_samples=1)

      idx = torch.cat((idx, idx_next), dim=1)

    return idx



model = BigramLanguageModel(vocab_size)
m = model.to(device)
optimizer = torch.optim.AdamW(m.parameters(), learning_rate)

print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')
batch_size = 32
for steps in range(max_iters):

  if steps % eval_interval == 0 or steps == max_iters - 1:
      losses = estimate_loss()
      print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batch("train")

  logits, loss = m(xb, yb)

  optimizer.zero_grad(set_to_none=True)

  loss.backward()

  optimizer.step()


print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=2000)[0].tolist()))